In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
%cd /content/drive/MyDrive/segment-geospatial/

/content/drive/MyDrive/segment-geospatial


In [ ]:
!ls

 100points_iou.csv					 masks.png
 100points_scores.csv					 masks.tif
 10k_random.csv						 mkdocs.yml
 annotations.png					 modeling
 annotations.tif					 paper
'Automatically generating object masks with SAM.ipynb'	'prompt_encoder.py '
 CITATION.cff						 README.md
 common.py						 requirements_dev.txt
 deneme2.png						 requirements_docs.txt
 deneme3.png						 requirements.txt
 deneme.png						 samgeo
 Dockerfile						'sam.py '
 docs							 satellite.tif
'image_encoder.py '					 segment_probabilities.csv
 __init__.py						 setup.cfg
 LICENSE						 setup.py
 MANIFEST.in						 tests
 mask2.tif						 transformer.py
'mask_decoder.py '					 tree_boxes.geojson


In [ ]:
!pip install leafmap
!pip install segment-geospatial
!pip install segment-anything
!pip install localtileserver

main.py

In [ ]:
from segment_anything import sam_model_registry, SamAutomaticMaskGenerator, SamPredictor
from samgeo.common import *
from samgeo.samgeo import *

sam = SamGeo(
    model_type="vit_h",
    checkpoint="sam_vit_h_4b8939.pth",
    sam_kwargs=None,
    automatic = False, # --> to use generate function set automatic param as True,
)                      # --> to use predict function set automatic param as False

In [ ]:
import ee
from datetime import datetime

ee.Authenticate()
ee.Initialize()

To authorize access needed by Earth Engine, open the following URL in a web browser and follow the instructions. If the web browser does not start automatically, please manually browse the URL below.

    https://code.earthengine.google.com/client-auth?scopes=https%3A//www.googleapis.com/auth/earthengine%20https%3A//www.googleapis.com/auth/devstorage.full_control&request_id=7gXxJSQGcv7YnpfXzg9ytXbRRyCw3FroCeXYcaqUt8M&tc=-WEnx1t4lmIdeFSZ3QapBQDCOL7OLArmrjhym-Ihb5I&cc=bveXipmM6DKXKXwlnPFUbdM7HUF8MtSQZrGpEgi6WPY

The authorization workflow will generate a code, which you should paste in the box below.
Enter verification code: 4/1AfJohXnr-MqiQkoGRd4PbfFDF6iLH_YE7PMJDFYfAmIKqpSxeL8pXVXdlE4

Successfully saved authorization token.


In [ ]:
# coordinate = [25.281610745652436, 54.691355848766996]
import pandas as pd

random_point_df = pd.read_csv('/content/drive/MyDrive/segment-geospatial/10k_random.csv')
coordinates = random_point_df[['lat', 'lon']].values.tolist()
# point = ee.Geometry.Point(coordinate)
start_date = '2022-01-01'
end_date = '2023-01-01'

In [ ]:
dataset = ee.ImageCollection('COPERNICUS/S2')

# filter according to time interval
filtered_dataset = dataset.filterDate(ee.Date(start_date), ee.Date(end_date))

# filter
# filtered_dataset = filtered_dataset.filterBounds(point)


Predicted IOUs

In [ ]:
import numpy as np
from PIL import Image
import requests
from io import BytesIO
import torch
import torchvision.transforms as transforms
import cv2
import tempfile

df = pd.DataFrame()

ious = []
for i in range(len(coordinates)):
  if i >= 21:
    print("Number of point: ", i)
    point = ee.Geometry.Point(coordinates[i])

    # filter
    filtered_dataset_ = filtered_dataset.filterBounds(point)

    flag = False
    for image in filtered_dataset_.toList(filtered_dataset_.size()).getInfo():
      image_date = datetime.utcfromtimestamp(image['properties']['system:time_start'] / 1000.0).strftime('%Y-%m-%d')
      print(f"Processing image for date: {image_date}")

      selected_image = ee.Image(image['id'])
      rgb_image = selected_image.select(['B4', 'B3', 'B2'])

      url = rgb_image.getThumbURL({'dimensions': 256, 'format': 'png'})

      response = requests.get(url)
      img = Image.open(BytesIO(response.content))

      # Resize the image to meet the library's requirements
      resized_image = cv2.resize(np.array(img), (256, 256))

      # Convert the NumPy array to a PyTorch tensor
      tensor_image = transforms.ToTensor()(resized_image)

      # Add a batch dimension (BCHW)
      tensor_image = tensor_image.unsqueeze(0) # (batch, channel, width, height) --> increase batch

      # create df
      if not flag:
        print(f"IOU Date: {image_date}")
        with tempfile.NamedTemporaryFile(suffix=".png", delete=False) as temp_img:
          temp_img_path = temp_img.name
          temp_img.close()
          transforms.ToPILImage()(tensor_image.squeeze()).save(temp_img_path) # tensor_image

          # Call sam.generate with the file path
          sam.generate(temp_img_path, output="masks.png", foreground=True, unique=True)

        pred_iou = sam.probability(axis="on", alpha=1, output="annotations.png")
        df = pd.concat([df, pd.DataFrame({'Date_'+ str(i): [image_date],  str(i) : [pred_iou]}),], ignore_index=True) # df = pd.concat([df, pd.DataFrame({'Date_'+ str(i): [image_date],  str(i) : [pred_iou]}),], ignore_index=True)

        os.remove(temp_img_path)
        flag = True

      if (image_date not in df['Date_' + str(i)].values):
        print(f"IOU Date: {image_date}")
        with tempfile.NamedTemporaryFile(suffix=".png", delete=False) as temp_img:
          temp_img_path = temp_img.name
          temp_img.close()
          transforms.ToPILImage()(tensor_image.squeeze()).save(temp_img_path)

          # Call sam.generate with the file path
          sam.generate(temp_img_path, output="masks.png", foreground=True, unique=True)

        pred_iou = sam.probability(axis="on", alpha=1, output="annotations.png")
        df = pd.concat([df, pd.DataFrame({'Date_'+ str(i): [image_date],  str(i) : [pred_iou]}),], ignore_index=True)

        # Remove the temporary image file
        os.remove(temp_img_path)

    ious.append(df)
    df = pd.DataFrame()
    if i != 0 and i % 10 == 0:
      save_df = pd.concat(ious, axis=1)
      save_df.to_csv('/content/drive/MyDrive/segment-geospatial/100points_iou.csv')

Scores

In [ ]:
import numpy as np
from PIL import Image
import requests
from io import BytesIO
import torch
import torchvision.transforms as transforms
import cv2
import tempfile

df = pd.DataFrame()

ious = []
for i in range(len(coordinates)):
  if i >= 21:
    print("Number of point: ", i)
    point = ee.Geometry.Point(coordinates[i])

    # filter
    filtered_dataset_ = filtered_dataset.filterBounds(point)

    flag = False
    for image in filtered_dataset_.toList(filtered_dataset_.size()).getInfo():
      image_date = datetime.utcfromtimestamp(image['properties']['system:time_start'] / 1000.0).strftime('%Y-%m-%d')
      print(f"Processing image for date: {image_date}")

      selected_image = ee.Image(image['id'])
      rgb_image = selected_image.select(['B4', 'B3', 'B2'])

      url = rgb_image.getThumbURL({'dimensions': 256, 'format': 'png'})

      response = requests.get(url)
      img = Image.open(BytesIO(response.content))

      # Resize the image to meet the library's requirements
      resized_image = cv2.resize(np.array(img), (256, 256))

      # Convert the NumPy array to a PyTorch tensor
      tensor_image = transforms.ToTensor()(resized_image)

      # Add a batch dimension (BCHW)
      tensor_image = tensor_image.unsqueeze(0) # (batch, channel, width, height) --> increase batch

      # create df
      if not flag:
        print(f"IOU Date: {image_date}")
        with tempfile.NamedTemporaryFile(suffix=".png", delete=False) as temp_img:
          temp_img_path = temp_img.name
          temp_img.close()
          transforms.ToPILImage()(tensor_image.squeeze()).save(temp_img_path)

          sam.set_image(temp_img_path)
          point_ = [[coordinates[i][1], coordinates[i][0]]]

          masks, scores, logits = sam.predict(point_coords=point_, return_results=True, multimask_output=False, output="mask2.tif")

          print("Scores: ", scores)

        df = pd.concat([df, pd.DataFrame({'Date_' + str(i): [image_date], str(i): [scores]}), ], ignore_index=True)

        os.remove(temp_img_path)

        flag = True

      if (image_date not in df['Date_' + str(i)].values):
        print(f"IOU Date: {image_date}")
        with tempfile.NamedTemporaryFile(suffix=".png", delete=False) as temp_img:
          temp_img_path = temp_img.name
          temp_img.close()
          transforms.ToPILImage()(tensor_image.squeeze()).save(temp_img_path)

          sam.set_image(temp_img_path)
          point_ = [[coordinates[i][1], coordinates[i][0]]]

          masks, scores, logits = sam.predict(point_coords=point_, return_results=True, multimask_output=False, output="mask2.tif")

          print("Scores: ", scores)

        df = pd.concat([df, pd.DataFrame({'Date_' + str(i): [image_date], str(i): [scores]}), ], ignore_index=True)

        os.remove(temp_img_path)

    ious.append(df)
    df = pd.DataFrame()
    if i != 0 and i % 10 == 0:
      save_df = pd.concat(ious, axis=1)
      save_df.to_csv('/content/drive/MyDrive/segment-geospatial/100points_scores2.csv')